In [1]:
import pandas as pd

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np

import os

import pdir

In [2]:
# Script inputs
    
FILE_PATH = "../orbsim/r4b_3d/ephemerides/"

PLANETS = ["earth",
           "mars"]

END_YEAR = "2020"

EXTRAPOLATION_POINTS = 9  # number of extrapolated points on line segnment between two table position values.
                          # e.g. 9 will give 0.1, 0.2, ..., 0.8, 0.9 on the segment between 0 and 1.

In [3]:
ephemerides_filename_dict = {}

for planet in PLANETS:
    ephemerides_filename_dict[planet] = "{}_2019-{}.csv".format(planet, END_YEAR)

ephemerides_filename_dict

{'earth': 'earth_2019-2020.csv', 'mars': 'mars_2019-2020.csv'}

In [4]:
# Change CWD of necessary
cwd = os.getcwd()
in_correct_cwd = 'code' + FILE_PATH[2:-1] == cwd[-30:] # Check if last part of cwd is '/code/orbsim/r4b_3d'

if not in_correct_cwd:    
    os.chdir(FILE_PATH)
    cwd = os.getcwd()

print(cwd)

/Users/gandalf/Dropbox/repositories/letomes/code/orbsim/r4b_3d/ephemerides


In [5]:
# Read CSV files into dict
ephemerides = {}
for body, csv_filename in ephemerides_filename_dict.items():
    ephemerides[body] = pd.read_csv(csv_filename, parse_dates=['date'])

pd.set_option("max_row", 5)
ephemerides    

{'earth':        day      MJD       date         r      theta         phi         x  \
 0      0.0  58484.0 2019-01-01  0.983311  66.939900  100.873526 -0.170672   
 1      1.0  58485.0 2019-01-02  0.983304  67.020209  101.977771 -0.187873   
 ..     ...      ...        ...       ...        ...         ...       ...   
 364  364.0  58848.0 2019-12-31  0.983318  66.849649   99.485272 -0.148996   
 365  365.0  58849.0 2020-01-01  0.983293  66.920303  100.591037 -0.166262   
 
             y         z  
 0    0.888496  0.385160  
 1    0.885562  0.383888  
 ..        ...       ...  
 364  0.891776  0.386587  
 365  0.889180  0.385462  
 
 [366 rows x 9 columns],
 'mars':        day      MJD       date         r       theta         phi         x  \
 0      0.0  58484.0 2019-01-01  1.453495   74.996659   39.051247  1.090280   
 1      1.0  58485.0 2019-01-02  1.454684   74.803063   39.609720  1.081506   
 ..     ...      ...        ...       ...         ...         ...       ...   
 364  36

In [6]:
# Linear extrapolation of all planet positions

extrapolated_ephemerides = {}

for body, eph in ephemerides.items():
    cols = eph.columns
    extrapolated_body = pd.DataFrame(columns=cols)
    extrapolated_body = extrapolated_body.append(eph.iloc[[0]])  # Add initial point

    N = len(eph) - 1  # number of intervals that needs extrapolation (0->1, 1->2, ...,)

    for n in range(N):  # add all points for day ]n, n+1]

        start_position_df = eph.iloc[[n]]
        end_position_df = eph.iloc[[n+1]]

        start_position_series = start_position_df.iloc[0]
        end_position_series = end_position_df.iloc[0]

        position_difference_series = end_position_series - start_position_series

        start_day = int(start_position_df['day'])

        for i in range(1,EXTRAPOLATION_POINTS+1):
            extrapolated_position_series = start_position_series + i / (EXTRAPOLATION_POINTS + 1) * position_difference_series
            extrapolated_position_df = pd.DataFrame(data=[extrapolated_position_series.values], columns=cols)
            extrapolated_body = extrapolated_body.append(extrapolated_position_df)

        extrapolated_body = extrapolated_body.append(end_position_df) # Add upper bound in current interval

    extrapolated_body.set_index('day', inplace=True)

    extrapolated_ephemerides[body] = extrapolated_body

In [7]:
extrapolated_ephemerides

{'earth':            MJD                date         r      theta         phi         x  \
 day                                                                             
 0.0    58484.0 2019-01-01 00:00:00  0.983311  66.939900  100.873526 -0.170672   
 0.1    58484.1 2019-01-01 02:24:00  0.983311  66.947931  100.983950 -0.172392   
 ...        ...                 ...       ...        ...         ...       ...   
 364.9  58848.9 2019-12-31 21:36:00  0.983296  66.913238  100.480461 -0.164535   
 365.0  58849.0 2020-01-01 00:00:00  0.983293  66.920303  100.591037 -0.166262   
 
               y         z  
 day                        
 0.0    0.888496  0.385160  
 0.1    0.888202  0.385032  
 ...         ...       ...  
 364.9  0.889440  0.385574  
 365.0  0.889180  0.385462  
 
 [3651 rows x 8 columns],
 'mars':            MJD                date         r       theta         phi  \
 day                                                                    
 0.0    58484.0 2019-01-01 00:

In [8]:
# # Write dicts to CSV files

# for body, filename in ephemerides_filename_dict.items():
#     basename, extension = filename.split('.')
#     extended_filename = "{}_extrapolated-{}.csv".format(basename,EXTRAPOLATION_POINTS)
#     extrapolated_ephemerides[body].to_csv(extended_filename)

In [23]:
extrapolated_ephemerides1 = extrapolated_ephemerides

In [27]:
extrapolated_ephemerides['earth']

,MJD,date,r,theta,phi,x,y,z
day,,,,,,,,
0.0,58484.0,2019-01-01 00:00:00,0.983311,66.939900,100.873526,-0.170672,0.888496,0.385160
0.1,58484.1,2019-01-01 02:24:00,0.983311,66.947931,100.983950,-0.172392,0.888202,0.385032
0.2,58484.2,2019-01-01 04:48:00,0.983310,66.955962,101.094375,-0.174112,0.887909,0.384905
0.3,58484.3,2019-01-01 07:12:00,0.983309,66.963993,101.204799,-0.175832,0.887616,0.384778
0.4,58484.4,2019-01-01 09:36:00,0.983308,66.972024,101.315224,-0.177552,0.887322,0.384651
...,...,...,...,...,...,...,...,...
364.6,58848.6,2019-12-31 14:24:00,0.983303,66.892042,100.148731,-0.159356,0.890219,0.385912
364.7,58848.7,2019-12-31 16:48:00,0.983301,66.899107,100.259308,-0.161082,0.889959,0.385799
364.8,58848.8,2019-12-31 19:12:00,0.983298,66.906172,100.369884,-0.162809,0.889699,0.385687


In [28]:
# extrapolated_ephemerides['earth'].set_index('day', inplace=True)
extrapolated_ephemerides['earth'].insert(0, 'day', extrapolated_ephemerides['earth'].index)

In [29]:
extrapolated_ephemerides['earth']

,day,MJD,date,r,theta,phi,x,y,z
day,,,,,,,,,
0.0,0.0,58484.0,2019-01-01 00:00:00,0.983311,66.939900,100.873526,-0.170672,0.888496,0.385160
0.1,0.1,58484.1,2019-01-01 02:24:00,0.983311,66.947931,100.983950,-0.172392,0.888202,0.385032
0.2,0.2,58484.2,2019-01-01 04:48:00,0.983310,66.955962,101.094375,-0.174112,0.887909,0.384905
0.3,0.3,58484.3,2019-01-01 07:12:00,0.983309,66.963993,101.204799,-0.175832,0.887616,0.384778
0.4,0.4,58484.4,2019-01-01 09:36:00,0.983308,66.972024,101.315224,-0.177552,0.887322,0.384651
...,...,...,...,...,...,...,...,...,...
364.6,364.6,58848.6,2019-12-31 14:24:00,0.983303,66.892042,100.148731,-0.159356,0.890219,0.385912
364.7,364.7,58848.7,2019-12-31 16:48:00,0.983301,66.899107,100.259308,-0.161082,0.889959,0.385799
364.8,364.8,58848.8,2019-12-31 19:12:00,0.983298,66.906172,100.369884,-0.162809,0.889699,0.385687


In [80]:
date = 3500
try:
    result = extrapolated_ephemerides['earth'].iloc[date]
except IndexError:
    print("Date out of bounds. Please choose date no later than {}-01-01, i.e. index lower than {}".format(END_YEAR,len(extrapolated_ephemerides['earth'])))
else:
    print(result)

day                      350
MJD                    58834
date     2019-12-17 00:00:00
r                   0.984083
theta                66.6798
phi                  83.9568
x                  0.0951394
y                   0.898668
z                   0.389569
Name: 350.0, dtype: object


In [ ]:
closest = extrapolated_ephemerides['earth'].index.get_loc(dt, method='nearest')
extrapolated_ephemerides['earth'].iloc[closest]

In [49]:
extrapolated_ephemerides['earth'].iloc[closest]

day                      1.2
MJD                  58485.2
date     2019-01-02 04:48:00
r                   0.983304
theta                67.0378
phi                  102.198
x                  -0.191302
y                    0.88492
z                    0.38361
Name: 1.2, dtype: object